In [39]:
import numpy as np
import tensorflow as tf

In [40]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import urllib

In [41]:
IRIS_TRAINING = "iris_training.csv"
IRIS_TRAINING_URL = "http://download.tensorflow.org/data/iris_training.csv"

IRIS_TEST = "iris_test.csv"
IRIS_TEST_URL = "http://download.tensorflow.org/data/iris_test.csv"

In [42]:
## Save datasets
if not os.path.exists(IRIS_TRAINING):
  raw = urllib.urlopen(IRIS_TRAINING_URL).read()
  with open(IRIS_TRAINING,'w') as f:
    f.write(raw)

if not os.path.exists(IRIS_TEST):
  raw = urllib.urlopen(IRIS_TEST_URL).read()
  with open(IRIS_TEST,'w') as f:
    f.write(raw)

In [43]:
training_set = tf.contrib.learn.datasets.base.load_csv_with_header(filename=IRIS_TRAINING, 
                                                                  target_dtype = np.int,
                                                                  features_dtype = np.float32)
test_set = tf.contrib.learn.datasets.base.load_csv_with_header(filename=IRIS_TEST, 
                                                                  target_dtype = np.int,
                                                                  features_dtype = np.float32)

In [44]:
# Datasets contain data and target in themselves, they are organized as named tuples
training_set.data

array([[ 6.4000001 ,  2.79999995,  5.5999999 ,  2.20000005],
       [ 5.        ,  2.29999995,  3.29999995,  1.        ],
       [ 4.9000001 ,  2.5       ,  4.5       ,  1.70000005],
       [ 4.9000001 ,  3.0999999 ,  1.5       ,  0.1       ],
       [ 5.69999981,  3.79999995,  1.70000005,  0.30000001],
       [ 4.4000001 ,  3.20000005,  1.29999995,  0.2       ],
       [ 5.4000001 ,  3.4000001 ,  1.5       ,  0.40000001],
       [ 6.9000001 ,  3.0999999 ,  5.0999999 ,  2.29999995],
       [ 6.69999981,  3.0999999 ,  4.4000001 ,  1.39999998],
       [ 5.0999999 ,  3.70000005,  1.5       ,  0.40000001],
       [ 5.19999981,  2.70000005,  3.9000001 ,  1.39999998],
       [ 6.9000001 ,  3.0999999 ,  4.9000001 ,  1.5       ],
       [ 5.80000019,  4.        ,  1.20000005,  0.2       ],
       [ 5.4000001 ,  3.9000001 ,  1.70000005,  0.40000001],
       [ 7.69999981,  3.79999995,  6.69999981,  2.20000005],
       [ 6.30000019,  3.29999995,  4.69999981,  1.60000002],
       [ 6.80000019,  3.

In [48]:
#Specifying features have real-value data
feature_columns = [tf.contrib.layers.real_valued_column("", dimension=4)]


# metrics which will be used by monitor
validation_metrics = {
    "accuracy":
        tf.contrib.learn.MetricSpec(
            metric_fn=tf.contrib.metrics.streaming_accuracy,
            prediction_key="classes"),
    "precision":
        tf.contrib.learn.MetricSpec(
            metric_fn=tf.contrib.metrics.streaming_precision,
            prediction_key="classes"),
    "recall":
        tf.contrib.learn.MetricSpec(
            metric_fn=tf.contrib.metrics.streaming_recall,
            prediction_key="classes")
}


# Debug monitor
validation_monitor = tf.contrib.learn.monitors.ValidationMonitor(
    test_set.data,
    test_set.target,
    every_n_steps=50,
    metrics=validation_metrics)

# Building a Deep Neural Network
classifier = tf.contrib.learn.DNNClassifier(feature_columns=feature_columns, 
                                           hidden_units = [10, 20, 10],
                                           n_classes = 3,
                                           model_dir="iris_model",
                                           config = tf.contrib.learn.RunConfig(save_checkpoints_secs=1))


Instructions for updating:
Monitors are deprecated. Please use tf.train.SessionRunHook.
INFO:tensorflow:Using config: {'_save_checkpoints_secs': 1, '_num_ps_replicas': 0, '_keep_checkpoint_max': 5, '_tf_random_seed': None, '_task_type': None, '_environment': 'local', '_is_chief': True, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff410f88b10>, '_tf_config': gpu_options {
  per_process_gpu_memory_fraction: 1.0
}
, '_task_id': 0, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_evaluation_master': '', '_keep_checkpoint_every_n_hours': 10000, '_master': ''}


In [49]:
def get_train_inputs():
    x = tf.constant(training_set.data)
    y = tf.constant(training_set.target)
    
    return x,y

In [50]:
classifier.fit(input_fn=get_train_inputs,
               steps=2000,
               monitors=[validation_monitor])

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Saving checkpoints for 4002 into iris_model/model.ckpt.
INFO:tensorflow:loss = 0.0318252, step = 4002
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will only accept input_fn.
Example conversion:
  est = Estimator(...) -> est = SKCompat(Estimator(...))
Instructions for updating:
Estimator is decoupled from Scikit Learn interface by moving into
separate class SKCompat. Arguments x, y and batch_size are only
available in the SKCompat class, Estimator will 

INFO:tensorflow:Validation (step 5202): loss = 0.138373, auc = 0.998333, global_step = 5173, recall = 1.0, precision = 1.0, accuracy = 0.966667
INFO:tensorflow:global_step/sec: 260.284
INFO:tensorflow:loss = 0.0284354, step = 5302
INFO:tensorflow:global_step/sec: 1268.04
INFO:tensorflow:loss = 0.0282323, step = 5402
INFO:tensorflow:global_step/sec: 1133.97
INFO:tensorflow:loss = 0.0280185, step = 5502
INFO:tensorflow:global_step/sec: 1219.99
INFO:tensorflow:loss = 0.0278256, step = 5602
INFO:tensorflow:global_step/sec: 1271.43
INFO:tensorflow:loss = 0.0276393, step = 5702
INFO:tensorflow:global_step/sec: 1136.96
INFO:tensorflow:loss = 0.0274417, step = 5802
INFO:tensorflow:global_step/sec: 1239.39
INFO:tensorflow:loss = 0.0272701, step = 5902
INFO:tensorflow:Saving checkpoints for 5954 into iris_model/model.ckpt.
INFO:tensorflow:Saving checkpoints for 6001 into iris_model/model.ckpt.
INFO:tensorflow:Loss for final step: 0.0274032.


DNNClassifier(params={'head': <tensorflow.contrib.learn.python.learn.estimators.head._MultiClassHead object at 0x7ff410f88710>, 'hidden_units': [10, 20, 10], 'feature_columns': (_RealValuedColumn(column_name='', dimension=4, default_value=None, dtype=tf.float32, normalizer=None),), 'embedding_lr_multipliers': None, 'optimizer': None, 'dropout': None, 'gradient_clip_norm': None, 'activation_fn': <function relu at 0x7ff45b9338c0>, 'input_layer_min_slice_size': None})

In [ ]:
def get_test_inputs():
    x = tf.constant(test_set.data)
    y = tf.constant(test_set.target)
    
    return x, y

accuracy_score = classifier.evaluate(input_fn =get_test_inputs, steps=1)['accuracy']

print("Test accuracy: {0:f}".format(accuracy_score))

In [ ]:
# Classify two new flower samples.
def new_samples():
  return np.array(
    [[6.4, 3.2, 4.5, 1.5],
     [5.8, 3.1, 5.0, 1.7]], dtype=np.float32)

predictions = list(classifier.predict(input_fn=new_samples))

print(
    "New Samples, Class Predictions:    {}\n"
    .format(predictions))